In [ ]:
import numpy as np
from scipy.linalg import solve

In [11]:
from numpy import pi, sin, exp
from numpy.linalg import cond, det
from scipy.linalg import norm, qr

n = 11 
A = np.zeros((n+1, n+1))
for i in range(n+1):
    for j in range(n+1):
        if j == 0:
            A[i,j] = 1
        elif i == 0:
            A[i,j] = 0
        else:
            A[i,j] = 1/(2*i+1)**j
sumA = A.sum()
print(f"sumA: {sumA}")

condA = cond(A)
print(f"condA: {condA}")

print("comment1 = 'yes'")

detA = det(A)
print(f"detA: {detA}")

print("comment2 = 'no'")

Q,R = qr(A)
diag5Q = np.diag(Q)[:5]
print(f"diag5Q: {diag5Q}")

x_pts = np.array([0, 1/3, 1/5, 1/7, 1/9])
y_pts = sin(10*pi*x_pts)/(100*exp(x_pts))
veca = solve(A[:5, :5], y_pts)
print(f"veca: {veca}")


sumA: 13.509935844715358
condA: 2.0778039513457235e+17
comment1 = 'yes'
detA: -7.584782657868772e-83
comment2 = 'no'
diag5Q: [-0.28867513 -0.77990597 -0.35925163  0.14627472 -0.40354541]
veca: [   0.            0.86037863  -15.50499537   82.13272647 -130.58607689]


In [12]:
from numpy import zeros, exp
f = lambda x : (x-1)**4 + exp(x)
g = lambda x : f(x) - 2
g_prime = lambda x : 4*(x-1)**3 + exp(x)
newton_step = lambda x : x - g(x)/g_prime(x)

x0 = 3
x1 = newton_step(x0)
x2 = newton_step(x1)
x3 = newton_step(x2)

tol = 10**(-8)
xn = x0
xe = None 
while True:
    xe = newton_step(xn)
    if abs(xe - xn) < tol:
        break
    xn = xe
invf2 = xe
print(f"invf2: {invf2}")

invf2: 0.6884238065328873


In [16]:
from numpy import ones, sqrt, pi, sin, linspace
from scipy.special import gamma

N = 9
h = 1/9
x = linspace(0, 1, N+1)
w = 2*np.ones(N+1)
w[0] = w[-1] = 1
f = (2-2*x**2)**1.5 * sqrt(x)
trapf = h/2 * w.T @ f
print(f"trapf: {trapf}")
errtrap = abs(trapf - (16/(15*pi**0.5)) * (gamma(0.75))**2)
print(f"errtrap: {errtrap}")

y = linspace(0,1, N+1)
F = np.zeros((N+1, N+1))
for i in range(N+1):
    for j in range(N+1):
        F[i,j] = sin(x[i]*y[j]) * sqrt(x[i]+2*y[j])

trap2dF = h**2/4 * w @ F @ w.T
print(f"trap2dF: {trap2dF}")


trapf: 0.8810226297155423
errtrap: 0.022671327398035235
trap2dF: 0.33498855671317784


In [24]:
from numpy import ones, linspace, loadtxt, pi, exp, cos, sin
from scipy.interpolate import CubicSpline
from scipy.fft import fft
from scipy.linalg import lstsq

tdata = [1,2,3,4,5,6,7,8,9,10]
ydata = [77.64,94.53,68.17,39.68,56.99,65.23,50.8,43.29,48.66,93.17]

teval = np.arange(1,10) + 0.3

spl = CubicSpline(tdata, ydata)
yspteval = spl(teval)
print(f"yspteval: {yspteval}")

poly = lambda t : np.array([1, t, exp(t), cos(pi*t/2)])
M = [poly(t) for t in tdata]
xLS = lstsq(M, ydata)[0]

yLSteval = np.array([poly(t) for t in teval]) @ xLS
print(f"yLSteval: {yLSteval}")

m = 10
w = fft(ydata)
a0 = w[0].real/m

a = np.zeros(5)
a[:4] = 2/m * w.real[1:5]
a[4] = w[5].real/m
print(a)
b = [3.9119, 1.7992, -0.4331, -4.9655, 0]

ydfteval = [a0 + np.sum([a[i]*cos(2*pi*(i+1)*t/m) + b[i]*sin(2*pi*(i+1)*t/m) for i in range(5)]) for t in teval]
print(f"ydfteval: {ydfteval}")

yspteval: [89.11551718 89.7353398  56.78286361 41.37721576 62.41078334 62.02855087
 47.29204316 42.80659648 56.36219093]
yLSteval: [83.44709974 85.27932684 59.24535957 47.4801628  63.76370156 66.27466822
 42.08570637 35.33575429 65.25214159]
[17.50437269 14.50796724 -7.93537269 -6.88896724 -3.364     ]
ydfteval: [94.80744577729573, 53.98808519494918, 42.043406834250746, 63.249650061961134, 60.52863068999574, 49.50790369246462, 39.145054674855146, 63.88337010733187, 92.44200996716202]


In [29]:
from numpy import diag, exp, ones, zeros, linspace, inf
from numpy.linalg import norm

def do(N=5):
    h = 1/N
    x = linspace(0,1,N+1)
    A = diag([-2] * (N-1)) + diag(ones(N-2), 1) + diag(ones(N-2), -1)
    B = diag(exp(-3*x[1:-1])) @ (diag(ones(N-2), 1) - diag(ones(N-2), -1))
    sumB = B.sum()
    C = diag(ones(N-1))
    R = zeros(N-1)
    R[-1] = 1/h**2 - 3/(2*h) * exp(-3*(N-1)*h)
    Uapprox = solve(-1/h**2 * A + 3/(2*h) * B + 9*C, R)
    return Uapprox, A, sumB, C

Uapprox, A, sumB, C = do()
print(sumB)
print(f"Uapprox: {Uapprox}")

u = lambda x : (1-exp(3*x))/(1-exp(3))
u40 = u(linspace(0,1,41)[1:-1])
u80 = u(linspace(0,1,81)[1:-1])
U40 = do(40)[0]
U80 = do(80)[0]
E1 = norm(U40 - u40, np.inf)
E2 = norm(U80 - u80, np.inf)
E = [E1, E2]
print(f"E: {E}")


0.45809368280461393
Uapprox: [0.04368579 0.12341851 0.2678357  0.52877524]
E: [6.063649591098175e-05, 1.5177439757541045e-05]
